Imports and Drive

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/gdrive')
#torch.cuda.current_device(),torch.cuda.device_count(),torch.cuda.is_available()
!pip install tiny-tokenizer flair
!pip install flair
import nltk
nltk.download('averaged_perceptron_tagger')

from flair.embeddings import XLNetEmbeddings
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.data import  Sentence, Token
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, \
    CharLMEmbeddings
from typing import List
import torch
import os
import re
from flair.embeddings import BertEmbeddings
from flair.embeddings import XLMRobertaEmbeddings
from flair.data import Sentence
from flair.embeddings import TransformerWordEmbeddings
import pandas as pd

Read File (should be a csv/exl with sent, pos, labels)


In [ ]:
import nltk
from nltk import word_tokenize
from nltk import StanfordTagger
import nltk
import pandas as pd
nltk.download("popular")

#df = read your file here 
words=df['word']
allPOS=[]
for word in words:
  if word=="--padding--":
    allPOS.append("-PAD-")
  elif str(word)=="nan":
    allPOS.append("--Nan--")
  elif word.isdigit():
    allPOS.append("-NUM-")
  else:  
    allPOS.append(nltk.pos_tag([word])[0][1])
df['pos']=pd.Series(allPOS)
#df.to_csv(save here if you want)

pick either options document embedding or sentence embedding 

In [ ]:
pip install sentence-transformers

Word Embedding:

[link to Flairs word embedding documentation](https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/TRANSFORMER_EMBEDDINGS.md), Hugging face link of all pretrained models https://huggingface.co/transformers/v2.3.0/pretrained_models.html

---



In [ ]:
from flair.data import Sentence
from flair.embeddings import TransformerWordEmbeddings

Document embedding :

 [https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/DOCUMENT_POOL_EMBEDDINGS.md](https://)

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')

# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([glove_embedding])
# create an example sentence
sentence = Sentence('The grass is green . And the sky is blue .')

# embed the sentence with our document embedding
document_embeddings.embed(sentence)


example

In [ ]:
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

# init embedding
embedding = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')
#embedding=SentenceTransformerDocumentEmbeddings('nli-bert-base-max-pooling')

# create a sentence
sentence = Sentence('The grass is green .')

# embed the sentence
embedding.embed(sentence)



My  Embedding (word and sentence)

In [ ]:
df2=  pd.DataFrame()
words=df['word']
labels=df['predicted_label']
POS=df['pos']
trueLabels=df['true_label']

# getting no  of sents
sents=0
for word in words:
  if word=='.':
    sents+=1

assert(len(words)==len(labels)==len(POS)) #sanity check

#for filling original test file
allWords=[]
allLabels=[]
currentSent=0
allSentID=[]
allPos=[]
allTrueLabels=[]
#for sentence embedding purposes, 
currentSentence=[]
embedding = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')
allSentenceEmbeddings=[]
mySentenceEmbeddingsDict={}


#for loop to fill in the original test file and sentence embedding as well, word-label-SentID-POS-sentenceEmbedding
for word,label,pos,tLabel in zip(words,labels,POS,trueLabels):
   if word=="--padding--" or str(word)=="nan":
    continue  
   else:  
     allWords.append(word)
     allLabels.append(label)
     allPos.append(pos)
     allSentID.append(currentSent)
     currentSentence.append(word) 
     allTrueLabels.append(tLabel)
     if word!=".": 
          allSentenceEmbeddings.append("")

   if word==".":
    currentSentence=" ".join(currentSentence)
    sentence=Sentence(currentSentence) 
    embedding.embed(sentence) 
    allSentenceEmbeddings.append(str(sentence.embedding)) 
    mySentenceEmbeddingsDict[currentSent]=str(sentence.embedding)
    currentSentence=[]
    currentSent+=1 

embedding = TransformerWordEmbeddings('bert-base-uncased') #or xlnet-base-cased or xlm-roberta-base
allWordEmbeddings=[]
allWordEmbeddingsTensors=[] 

for word,label,pos in zip(words,labels,POS):
   if word=="--padding--" or str(word)=="nan": 
    continue  
   else:  
     currentSentence.append(word)

   if word==".":
    currentSentence=" ".join(currentSentence)
    sentence=Sentence(currentSentence, use_tokenizer=False) 
    sentTokens=[]
    embedding.embed(sentence)
    for token in sentence:

      sentTokens.append((str(token).split())[-1])
      allWordEmbeddings.append(str(token.embedding))
      allWordEmbeddingsTensors.append(token.embedding)
    sentTokens=" ".join(sentTokens)
    currentSentence=[]
    sentTokens=[]
    currentSent+=1 


Utlize output from NER model; if you want to use in fusion process

In [ ]:
df['finalSentID']=df['originalSentID']
df['FinalWord']=df['NERword']

Ner1=df['NERpredicted1']
Ner2=df['NERpredicted2']
NerFinal=[]

for n1,n2 in zip(Ner1,Ner2):
  if n1=='O':
    NerFinal.append(n2)
  else:
    NerFinal.append(n1)  

df['finalNER']=pd.Series(NerFinal)
df['finalOIELabel']=df['originalLabels']
df['finalPOS']=df['originalPOS']

df.to_csv("/content/gdrive/MyDrive/my Personal work/Open-CyKG/MalwareDB_dataset_csv_exl/OIEoutput(10Feb).csv", index=False)# float_format="%.10g")


--> create CSV output to use with NEO4J or your choice of Graph software

In [ ]:
#imports and read file
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
# df=read your file here
w1=allWordEmbeddingsTensors[0]
w2=allWordEmbeddingsTensors[1]
w1Plusw2 = w1.add(w2)
avg=w1Plusw2/2
#df3=read file here

# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t,l) for w, p, t, l in zip(s['FinalWord'].values.tolist(),
                                                       s['finalPOS'].values.tolist(),
                                                       s['finalNER'].values.tolist(),
                                                       s['finalTrueLabel'].values.tolist())] #injy
        self.grouped = self.df.groupby("finalSentID").apply(agg) #injy
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent +=1
            return s
        except:
            return None
getter = sentence(df3) 
sentences = [" ".join([str(s[0]) for s in sent]) for sent in getter.sentences]
#sentence with its pos and tag.
import collections
getter = sentence(df3) #injy

sent = getter.get_text()

sentences = getter.sentences

#declare all 3 dataframes that will represent the 3 final CSV files(tables)

dfRel=pd.DataFrame()
dfE1=pd.DataFrame()
dfE2=pd.DataFrame()


#Colomns to fill in the table
col_E1_ID=[]
col_E2_ID=[]
col_NameRel=[]
col_NameE1=[]
col_NameE2=[]
col_NER_Rel=[]
col_POS_Rel=[]
col_NER_E1=[]
col_POS_E1=[]
col_NER_E2=[]
col_POS_E2=[]
col_E1Rel_ID=[] 
col_E2Rel_ID=[]

allEntity1Dict={} 
allEntity2Dict={} 

sentID=0
for s in sentences:
    seenTriple=set()
    seen2Ent=set()
    seenE1ID=set() 
    seenE2ID=set() 
    OIELabels=[s[n][3] for n in range(len(s))]
    sentWord=[s[n][0] for n in range(len(s))]
    sentNER=[s[n][2] for n in range(len(s))]
    sentPOS=[s[n][1] for n in range(len(s))]
    
    e1=[]
    e2=[]
    rel=[]
    Labels=#Add list of labels here
    i=0
    eID=sentID 
    while i<len(s):
      label=OIELabels[i]
      if label=='O':
        i+=1
        continue
      if 'Action' in label or 'Modifier' in label:
        rel=[]
        if 'B' in label: 
            rel.append(str(i))
            i+=1
            while (OIELabels[i] in [Labels]) and i < len(s):
              rel.append(str(i))
              i+=1
      if 'Entity' in label:
        if 'B' in label and not e1: 
          e1.append(str(i))
          e1ID=round(eID+0.1,1)  
          eID=round(eID+0.1,1)  
          i+=1
          while OIELabels[i]=='I-Entity' and i < len(s):
            e1.append(str(i))
            i+=1
          laste1=1
          
        elif 'B' in label and not e2:
          e2.append(str(i))
          e2ID=round(eID+0.1,1) 
          eID=round(eID+0.1,1)  
          i+=1
          while  i < len(s) and OIELabels[i]=='I-Entity':
            e2.append(str(i))
            i+=1
          laste1=0
        elif e1 and e2 and laste1: 
          e2=[]
        elif e1 and e2 and not laste1: 
          e1=[]

      if e1 and e2 and rel:
          all3=e1+rel+e2
          all3="".join(all3)
          all3V2=e2+rel+e1
          all3V2="".join(all3V2)
          TwoEnt=e1+e2  
          TwoEnt="".join(TwoEnt)

          if all3 not in seenTriple and all3V2 not in seenTriple and TwoEnt not in seen2Ent:
                triple={'e1':e1, 'rel': rel, 'e2':e2}
                seenTriple.add(all3)
                seenTriple.add(all3V2)
                seen2Ent.add(TwoEnt)
                for key in triple:
                  words=POS=NER=""
                  if key=='e1' :
                    col_E1Rel_ID.append(e1ID)
                    if e1ID not in seenE1ID: col_E1_ID.append(e1ID)
                    
                  elif key=='e2':
                    col_E2Rel_ID.append(e2ID)
                    if  e2ID not in seenE2ID: col_E2_ID.append(e2ID) 

                  for value in triple[key]:
                    words+=sentWord[int(value)] 
                    words+=" "
                    NER+=sentNER[int(value)] 
                    NER+=" "
                    POS+=sentPOS[int(value)] 
                    POS+=" "

                  if key=='e1' and e1ID not in seenE1ID: 
                    col_NameE1.append(words)
                    col_NER_E1.append(NER) 
                    col_POS_E1.append(POS) 
                    seenE1ID.add(e1ID)
                  elif key=='e2' and e2ID not in seenE2ID: 
                    col_NameE2.append(words)
                    col_NER_E2.append(NER) 
                    col_POS_E2.append(POS) 
                    seenE2ID.add(e2ID)
                  elif key=='rel': 
                    col_NameRel.append(words)
                    col_NER_Rel.append(NER) 
                    col_POS_Rel.append(POS) 

    sentID+=1

In [ ]:
myE1Dict={}
idx=0
toDelE1_idx=set()
updateE1_inRelDict={}

for word,id in zip(col_NameE1,col_E1_ID):
     if word in myE1Dict: 
        toDelE1_idx.add(idx) 
        updateE1_inRelDict[id]=myE1Dict[word] 
     else:
       myE1Dict[word]=id
     idx+=1

col_E1_ID=[v for idx,v in enumerate(col_E1_ID) if idx not in toDelE1_idx]
col_NameE1=[v for idx,v in enumerate(col_NameE1) if idx not in toDelE1_idx]
col_NER_E1 =[v for idx,v in enumerate(col_NER_E1) if idx not in toDelE1_idx]
col_POS_E1=[v for idx,v in enumerate(col_POS_E1) if idx not in toDelE1_idx]

for idx,v in enumerate(col_E1Rel_ID):
     if v in updateE1_inRelDict:
       col_E1Rel_ID[idx]=updateE1_inRelDict[v]

myE2Dict={}
idx=0
toDelE2_idx=set()
updateE2_inRelDict={}

#step 1: identify entity appearning twice or more
for word,id in zip(col_NameE2,col_E2_ID):
     if word in myE2Dict:
        toDelE2_idx.add(idx) 
        updateE2_inRelDict[id]=myE2Dict[word] 
     else:
       myE2Dict[word]=id
     idx+=1

#step2: now remove indecees in toDelE1_idx from all Entity 1 table coloumns:

col_E2_ID=[v for idx,v in enumerate(col_E2_ID) if idx not in toDelE2_idx]
col_NameE2=[v for idx,v in enumerate(col_NameE2) if idx not in toDelE2_idx]
col_NER_E2 =[v for idx,v in enumerate(col_NER_E2) if idx not in toDelE2_idx]
col_POS_E2=[v for idx,v in enumerate(col_POS_E2) if idx not in toDelE2_idx]


#step3 : now Entity1 col in Relation table still has the old entry of the idx, update it from updateE1_inRelDict:
for idx,v in enumerate(col_E2Rel_ID):
     if v in updateE2_inRelDict:
       col_E2Rel_ID[idx]=updateE2_inRelDict[v]


 HAC
Create clusters for E1 [link ref CESI github](https://github.com/IS5882/cesi/blob/master/src/cluster.py)

In [ ]:
from helper import *
from joblib import Parallel, delayed
import numpy as np, time, random, pdb, itertools

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
n=len(allE1AvgEmbTensors)
m=768
X 	= np.empty((n, m), np.float32)


for i in range(len(allE1AvgEmbTensors)): 
			X[i, :] = allE1AvgEmbTensors[i]

dist 	  = pdist(X, 	  metric='cosine')
clust_res = linkage(dist, method='complete') 
labels    = fcluster(clust_res, t=.4329, criterion='distance') - 1
clusters  = [[] for i in range(max(labels) + 1)]
for i in range(len(labels)): 
			clusters[labels[i]].append(i)


Entity Representive and evaluation

In [ ]:
import collections
weights=collections.Counter()

for s in sentences:
  sentWord=[s[n][0] for n in range(len(s))]
  POS=[s[n][1] for n in range(len(s))]
  
  for word,pos in zip(sentWord,POS):
      weights[word]+=1

ent2freq=weights 
final_res = dict()
clusterNo=0
cesi_clust2ent_mod2={} 
index_to_word={}
for cluster in clusters:
      rep, max_freq = cluster[0], -1
      score, maxScore=0, 0
      wordList=[]
      for ent in cluster:
        words=col_NameE1[ent]
        wordList.append(words)
        index_to_word[ent]=words#
        for w in words:
          score+=ent2freq[w]
        if score > maxScore:
          maxScore, rep = score, ent
      final_res[rep] = cluster
      cesi_clust2ent_mod2[rep]= wordList
      clusterNo+=1
final_res_e1=final_res
index_e1=[]
words_e1=[]
for k,v in index_to_word.items():
    index_e1.append(k)
    words_e1.append(v)
    
#Gold standard eval
def invertDic(my_map, struct = 'o2o'):
	inv_map = {}
	if struct == 'o2o':				
		for k, v in my_map.items():
			inv_map[v] = k
	elif struct == 'm2o':			
		for k, v in my_map.items():
			inv_map[v] = inv_map.get(v, [])
			inv_map[v].append(k)
	elif struct == 'm2ol':				
		for k, v in my_map.items():
			for ele in v:
				inv_map[ele] = inv_map.get(ele, [])
				inv_map[ele].append(k)
	elif struct == 'm2os':
		for k, v in my_map.items():
			for ele in v:
				inv_map[ele] = inv_map.get(ele, set())
				inv_map[ele].add(k)
	return inv_map


import itertools, sys

def macroPrecision(C_clust2ele, E_ele2clust):
	num_prec = 0
	for _, cluster in C_clust2ele.items():
		isFirst = True
		res = set()
		for ele in cluster:
			if ele not in E_ele2clust: 
				continue
			if isFirst:
				res = E_ele2clust[ele]
				isFirst = False
				continue
			res = res.intersection(E_ele2clust[ele])
		if   len(res) == 1: num_prec += 1
	if len(C_clust2ele) == 0: return 0
	return float(num_prec) / float(len(C_clust2ele))

def microPrecision(C_clust2ele, E_ele2clust):
	num_prec = 0
	total = 0

	for _, cluster in C_clust2ele.items():
		freq_map = {}
		total += len(cluster)

		for ent in cluster:
			if ent not in E_ele2clust: 
				continue
			for ele in E_ele2clust[ent]:
				freq_map[ele] = freq_map.get(ele, 0)
				freq_map[ele] += 1
		max_rep = 0
		for k, v in freq_map.items(): max_rep = max(max_rep, v)

		num_prec += max_rep

	if total == 0: return 0
	return float(num_prec) / float(total)

def pairPrecision(C_clust2ele, E_ele2clust):
	num_hit = 0
	num_pairs = 0

	for _, cluster in C_clust2ele.items():
		all_pairs = list(itertools.combinations(cluster, 2))
		num_pairs += len(all_pairs)

		for e1, e2 in all_pairs:
			if e1 not in E_ele2clust or e2 not in E_ele2clust:
				continue

			res = E_ele2clust[e1].intersection(E_ele2clust[e2])
			if   len(res) == 1: num_hit += 1

	if num_pairs == 0: return 0
	return float(num_hit) / float(num_pairs)

def pairwiseMetric(C_clust2ele, E_ele2clust, E_clust2ent):
	num_hit = 0
	num_C_pairs = 0
	num_E_pairs = 0

	for _, cluster in C_clust2ele.items():
		all_pairs = list(itertools.combinations(cluster, 2))
		num_C_pairs += len(all_pairs)

		for e1, e2 in all_pairs:
			if e1 in E_ele2clust and e2 in E_ele2clust and len(E_ele2clust[e1].intersection(E_ele2clust[e2])) > 0: num_hit += 1

	for rep, cluster in E_clust2ent.items(): 
		num_E_pairs += len(list(itertools.combinations(cluster, 2)))

	if num_C_pairs == 0 or num_E_pairs == 0: 
		return 1e-6, 1e-6

	return float(num_hit) / float(num_C_pairs), float(num_hit) / float(num_E_pairs)


def calcF1(prec, recall):
	if prec + recall == 0: return 0
	return 2 * (prec * recall) / (prec + recall)

def microF1(C_ele2clust, C_clust2ele, E_ele2clust, E_clust2ent):
	micro_prec 	= microPrecision(C_clust2ele, E_ele2clust)
	micro_recall 	= microPrecision(E_clust2ent, C_ele2clust)
	micro_f1	= calcF1(micro_prec, micro_recall)
	return micro_f1

def macroF1(C_ele2clust, C_clust2ele, E_ele2clust, E_clust2ent):
	macro_prec 	= macroPrecision(C_clust2ele, E_ele2clust)
	macro_recall 	= macroPrecision(E_clust2ent, C_ele2clust)
	macro_f1	= calcF1(macro_prec, macro_recall)
	return macro_f1

def pairF1(C_ele2clust, C_clust2ele, E_ele2clust, E_clust2ent):
	pair_prec,pair_recall = pairwiseMetric(C_clust2ele, E_ele2clust, E_clust2ent)
	pair_f1		= calcF1(pair_prec, pair_recall)
	return pair_f1

def evaluate(C_ele2clust, C_clust2ele, E_ele2clust, E_clust2ent):
	macro_prec 	= macroPrecision(C_clust2ele, E_ele2clust)
	macro_recall 	= macroPrecision(E_clust2ent, C_ele2clust)
	macro_f1	= calcF1(macro_prec, macro_recall)

	micro_prec 	= microPrecision(C_clust2ele, E_ele2clust)
	micro_recall 	= microPrecision(E_clust2ent, C_ele2clust)
	micro_f1	= calcF1(micro_prec, micro_recall)

	pair_prec,pair_recall = pairwiseMetric(C_clust2ele, E_ele2clust, E_clust2ent)
	pair_f1		= calcF1(pair_prec, pair_recall)

	return {
		'macro_prec': 	round(macro_prec, 	4),
		'macro_recall':	round(macro_recall, 	4),
		'macro_f1':	round(macro_f1, 	4),

		'micro_prec': 	round(micro_prec, 	4),
		'micro_recall':	round(micro_recall, 	4),
		'micro_f1':	round(micro_f1, 	4),

		'pair_prec': 	round(pair_prec, 	4),
		'pair_recall':	round(pair_recall, 	4),
		'pair_f1':	round(pair_f1, 		4),
	}
#df_gold = read goldset
_id=df_gold['Entity1_ID']
true_lnk_sub2=df_gold['true_link_subject']
myTrueLink={k:v for k,v in zip(_id,true_lnk_sub2) }
myTrueLink2={}
count=0
for k,v in zip(_id,true_lnk_sub2):
    if k in myTrueLink2:
      index=count
      k=k+0.01
      newK=k
    myTrueLink2[k]=v
    count+=1
true_lnk_sub=[]
for key in sorted(myTrueLink2):
  true_lnk_sub.append(myTrueLink2[key])

triples_list=[]
dfE1=pd.read_csv("/content/gdrive/MyDrive/my Personal work/Open-CyKG/MalwareDB_dataset_csv_exl/KG/Ent1(12March2021).csv")

e1Names=dfE1E['Name']
Entity1_ID=dfE1E['Entity1_ID']
_id=sorted(myTrueLink2)
assert(len(e1Names)==len(_id)==len(true_lnk_sub))
for triple_unq, id, goldClust in zip(e1Names,_id,true_lnk_sub):
      trp={}
      trp['triple_unique']=triple_unq
      trp['true_sub_link']=goldClust
      triples_list.append(trp)

from collections import defaultdict as ddict
''' Ground truth clustering: from CESI main '''
true_ent2clust = ddict(set)
for trp in triples_list:
			sub_u = trp['triple_unique'] #i think its the subjet ie: entity word (Name)
			true_ent2clust[sub_u].add(trp['true_sub_link']) #true_link_subject
true_clust2ent = invertDic(true_ent2clust, 'm2os')  


#embedding evaluation from CESI main np_eval function
ent_clust=final_res_e1
cesi_clust2ent = {}
for rep, cluster in ent_clust.items(): 
  cesi_clust2ent[rep] = set(cluster)
cesi_ent2clust = invertDic(cesi_clust2ent, 'm2os')

word_to_index=invertDic(index_to_word, 'o2o')
true_clust2ent_mod2=ddict(list)

for k in true_clust2ent:
  first=1
  for v in true_clust2ent[k]:
    if first:
      key=word_to_index[v]
    first=0
    true_clust2ent_mod2[key].append(v)

cesi_ent2clust_mod2 = invertDic(cesi_clust2ent_mod2, 'm2os')
true_ent2clust_mod2 = invertDic(true_clust2ent_mod2, 'm2os')  

eval_results = evaluate(cesi_ent2clust_mod2, cesi_clust2ent_mod2, true_ent2clust_mod2,true_clust2ent_mod2)

print('Macro P: {}, Micro P: {}, Pairwise P: {}'.format(eval_results['macro_prec'], eval_results['micro_prec'], eval_results['pair_prec']))

print('Macro R: {}, Micro R: {}, Pairwise R: {}'.format(eval_results['macro_recall'], eval_results['micro_recall'], eval_results['pair_recall']))

print('Macro F1: {}, Micro F1: {}, Pairwise F1: {}'.format(eval_results['macro_f1'], eval_results['micro_f1'], eval_results['pair_f1']))
print('CESI: #Clusters: %d, #Singletons %d'    % (len(cesi_clust2ent_mod2), 	len([1 for _, clust in cesi_clust2ent_mod2.items()    if len(clust) == 1])))
print('Gold: #Clusters: %d, #Singletons %d \n' % (len(true_clust2ent_mod2),  len([1 for _, clust in true_clust2ent_mod2.items() if len(clust) == 1])))


Macro_r_e1=eval_results['macro_recall']
Macro_p_e1=eval_results['macro_prec']
Macro_f_e1=eval_results['macro_f1']

Micro_r_e1=eval_results['micro_recall']
Micro_p_e1=eval_results['micro_prec']
Micro_f_e1=eval_results['micro_f1']

pw_r_e1=eval_results['pair_recall']
pw_p_e1=eval_results['pair_prec']
pw_f_e1=eval_results['pair_f1']